In [ ]:
# Install dependencies
!pip install transformers
!pip install torch

In [ ]:
# Import dependencies
import torch
from transformers import BertTokenizer, BertModel
from transformers import pipeline

In [ ]:
# Check if a GPU is available
torch.cuda.is_available()